In [15]:
import os

prj = "./main.go"
# prj_dir, prj_file_tail = os.path.split(prj)
# prj_name = prj_file_tail.split(".")[0]
with open(prj, "r") as f:
     lines = f.readlines()
lines = [s.strip('\n') for s in lines]
lines

['package main',
 '',
 'import (',
 '\t"fmt"',
 '',
 '\t"github.com/USACE/go-consequences/compute"',
 '\t"github.com/USACE/go-consequences/hazardproviders"',
 '\t"github.com/USACE/go-consequences/resultswriters"',
 '\t"github.com/USACE/go-consequences/structureprovider"',
 ')',
 '',
 'func main() {',
 '\tfmt.Println("Start")',
 '',
 '\t//define flood hazard, provide the file path to your desired flood',
 '\thazard, e := hazardproviders.Init("./data/flood_here.tif")',
 '\tif e != nil {',
 '\t\tpanic(e)',
 '\t}',
 '\tdefer hazard.Close()',
 '',
 '\t//define structure inventory, provide file path to structure inventory',
 '\tinventory, b := structureprovider.InitSHP("./data/inventory_here.shp")',
 '\tif b != nil {',
 '\t\tpanic(b)',
 '\t}',
 '',
 '\t//define result file, provide file path where you want results to be saved',
 '\t// results, c := resultswriters.InitGpkResultsWriter("./output/result_file.gpkg", "event_results")',
 '\tresults, c := resultswriters.InitGpkResultsWriter_Project

In [16]:
# Get Hazard Layer (WSE geotif from HEC-RAS)
# splitting by double quotes because GoLang always uses double quotes for defining strings.
hazard_layer = [s for s in lines if "hazardproviders.Init" in s][0].split('"')[1]

In [17]:
# Get Structure Inventory Layer
structure_inventory_layer = [s for s in lines if "structureprovider.Init" in s][0].split('"')[1]

In [19]:
# Get Projected Results Layer
try:
    results_layer = [s for s in lines if "resultswriters.InitGpkResultsWriter_Projected" in s][0].split('"')[1]
    # Getting the projection from the last parameter of the InitGpkResultsWriter_Projected function call.
    results_projection = [s for s in lines if "resultswriters.InitGpkResultsWriter_Projected" in s][0].split(',')[-1].split(')')[0]
except:
    # Get Results Layer that assumes 4326 projection
    results_layer = [s for s in lines if "resultswriters.InitGpkResultsWriter" in s][0].split('"')[1]
    results_projection = "4326"